In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

# import os
# for dirname, _, filenames in os.walk('/kaggle/working'):
#     for filename in filenames[0:1]:
#         print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All"
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session


In [ ]:
# scarico i dati
!git clone https://github.com/yumoxu/stocknet-dataset.git

Cloning into 'stocknet-dataset'...
fatal: unable to access 'https://github.com/yumoxu/stocknet-dataset.git/': Could not resolve host: github.com


In [ ]:
%%capture
# installo pacchetti
!pip install transformers tweetnlp emot==2.1 spacymoji emoji==2.6.0

In [ ]:
# import stock data

# Import the libraries
import pandas as pd
import os
from tqdm.auto import tqdm
import multiprocessing as mp
from tqdm.contrib.concurrent import process_map

import numpy as np # linear algebra

tqdm.pandas()

pd.set_option('display.max_colwidth', 240)

# Define a function to get all the files in a folder and its subfolders
def get_files(path):
  # Initialize an empty list to store the files
  files = []
  # Loop over all the files and folders in the given path
  for file in os.listdir(path):
    # If the file or folder is a file, add it to the list of files
    if os.path.isfile(os.path.join(path, file)):
        files.append(os.path.join(path, file))
    # If the file or folder is a folder, call the function recursively on its path and add the results to the list of files
    elif os.path.isdir(os.path.join(path, file)):
        files.extend(get_files(os.path.join(path, file)))
  # Return the list of files as output
  return files


def read_json(file):
    df = pd.read_json(
        file,orient='records',lines=True
    )[["created_at","user","text"]]
    # Aggiungi due nuove colonne con il nome dello stock e la data, ottenuti dal percorso del file
    stock_name = os.path.basename(os.path.dirname(file))
    date = os.path.splitext(os.path.basename(file))[0]
    df["stock_name"] = stock_name
    df["date"] = date
    df["user"] = df["user"].apply(lambda x: x["id"])
    return df

def read_csv(file):
    df = pd.read_csv(
        file
    )
    stock_name = os.path.splitext(os.path.basename(file))[0]
    df["stock_name"] = stock_name
    return df

In [ ]:
# Crea un pool di processi con il numero di core della tua macchina
pool = mp.Pool(mp.cpu_count())

# Ottieni la lista dei file da leggere
files = get_files("/kaggle/working/stocknet-dataset/tweet/raw")

# Usa pool.map per applicare la funzione read_json a tutti i file in parallelo
dataframes = process_map(read_json, files, max_workers=mp.cpu_count(), chunksize=146)


# Chiudi il pool e rilascia le risorse
pool.close()

# Unisci i dataframe in uno solo
data = pd.concat(dataframes)


In [ ]:
data["date"] = pd.to_datetime(data["date"], format="%Y-%m-%d")


In [ ]:
data["created_at"] = pd.to_datetime(data["created_at"])


In [ ]:
data=data.reset_index(drop=True)
print(data.shape)
data.head()

In [ ]:
data.sample(5)

In [ ]:
data.groupby("stock_name").date.agg(["min", "max", "median", "count"])

In [ ]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer
import nltk
from nltk.corpus import stopwords
from nltk.stem import PorterStemmer

# Importa le librerie necessarie
import emoji
import emot
import re

import torch


In [ ]:
# Scarica il file slang.txt
!wget https://raw.githubusercontent.com/rishabhverma17/sms_slang_translator/master/slang.txt

# Apri il file slang.txt e leggi le sue righe
with open("slang.txt") as f:
    lines = f.readlines()

# Crea un dizionario vuoto
slang_dict = {}

# Aggiungi le coppie slang-traduzione al dizionario
for line in lines:
    # Rimuovi eventuali spazi o caratteri speciali dalla riga
    line = line.strip()
    # Dividi la riga in due parti usando l'uguale (=) come separatore
    parts = line.replace("\t", "=").split("=")
    # Assegna la prima parte alla variabile slang e la seconda parte alla variabile translation
    slang = parts[0]
    translation = parts[1]
    # Aggiungi la coppia slang-traduzione al dizionario
    slang_dict[slang] = translation

# Definisci una funzione per tradurre gli slang
def translate_slang(text):
  # Dividi il testo in parole
  words = text.split()
  # Crea una lista vuota
  translated_words = []
  # Itera sulle parole del testo
  for word in words:
    # Controlla se la parola è presente nel dizionario
    if word in slang_dict:
      # Aggiungi alla lista la traduzione dello slang
      translated_words.append(slang_dict[word])
    else:
      # Aggiungi alla lista la parola stessa
      translated_words.append(word)
  # Unisci le parole della lista in una stringa
  translated_text = " ".join(translated_words)
  # Restituisci la stringa come risultato della funzione
  return translated_text

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = AutoModelForSequenceClassification.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment-latest"
)
model.to(device)
tokenizer = AutoTokenizer.from_pretrained(
    "cardiffnlp/twitter-roberta-base-sentiment-latest",
    use_fast=False
)


In [ ]:

def preprocess(text):
    new_text = []
    for t in text.split(" "):
        t = '@user' if t.startswith('@') and len(t) > 1 else t
        t = 'http' if t.startswith('http') else t
        new_text.append(t)
    return " ".join(new_text)


# emot_obj = emot.core.emot()

# Definisci una funzione che normalizza il testo dei tweet
def normalize_tweet(text):
    # Rimuovi gli username e i link usando la funzione preprocess degli autori del modello
    text = preprocess(text)
    # Sostituisci le emoji con le parole che ne esprimono il significato usando la libreria emoji
    text = emoji.demojize(text, delimiters=("  ", "  "))
    # Traduci gli acronimi e lo slang in inglese standard usando il dizionario
    text = translate_slang(text)
    # Rimuovi eventuali spazi multipli o caratteri speciali usando delle espressioni regolari
    text = re.sub(r"\s+", " ", text)
    text = re.sub(r"[^\w\s@]", "", text)
    # removes line breaks
    text = re.sub(r"\n", " ", text)
    text = text.lower()
    # Restituisci il testo normalizzato
    return text


def predict_sentiment(text):
    # Tokenizza il testo usando il tokenizer
    encoded_input = tokenizer(text, return_tensors='pt')
    encoded_input = encoded_input.to(device)
    # Fai la predizione usando il modello
    output = model(**encoded_input)
    # Ottieni il valore della logit più alta
    scores = output[0][0].detach()
    scores = scores.cpu().numpy()
    scores = np.exp(scores) / np.exp(scores).sum(-1)
    # Mappa i valori delle logit alle etichette del sentimento
    labels = ['NEGATIVE', 'NEUTRAL', 'POSITIVE', 'MIXED']
    ranking = np.argsort(scores)
    ranking = ranking[::-1]
    # Restituisci l'etichetta e lo score del sentimento più probabile
    return (labels[ranking[0]], scores[ranking[0]])

In [ ]:
normalize_tweet("@izokipla $GOOG is part of the #MG25 index. Interested in your opinion. https://t.co/jLSGu4l8m3	")

In [ ]:
# puliamo i tweet
data['cleaned_text'] = data[['text']].progress_apply(
    lambda row: normalize_tweet(row['text']), axis=1)



In [ ]:
# calcoliuamo sentiment e score
data[['sentiment', 'score']] = data[['cleaned_text']].progress_apply(
    lambda row: predict_sentiment(row['cleaned_text']), axis=1, result_type='expand')


In [ ]:
from pathlib import Path
Path("/kaggle/working/data").mkdir(parents=True, exist_ok=True)
data.to_csv("/kaggle/working/data/DB_TWEET_TOT_RIPULITO.csv", index=False)

In [ ]:
# Create a namedtuple for named aggregation
size = pd.NamedAgg(column="cleaned_text", aggfunc="size")
max_len = pd.NamedAgg(column="cleaned_text", aggfunc=lambda x: x.str.len().max())
mean_len = pd.NamedAgg(column="cleaned_text", aggfunc=lambda x: x.str.len().mean())
mean_score = pd.NamedAgg(column="score", aggfunc="mean")
min_score = pd.NamedAgg(column="score", aggfunc="min")
max_score = pd.NamedAgg(column="score", aggfunc="max")
def flatten(df_):
    cols = ["_".join(cs) for cs in df_.columns.to_flat_index()]
    df_.columns = cols
    return df_

# Apply the named aggregations to the groupby object
result = data.groupby(["stock_name", data["created_at"].dt.date, "sentiment"]).agg(
    size=size,
    max_len=max_len,
    mean_len=mean_len,
    mean_score=mean_score,
    min_score=min_score,
    max_score=max_score
).unstack().pipe(flatten).reset_index()

result

In [ ]:
from pathlib import Path
Path("/kaggle/working/data").mkdir(parents=True, exist_ok=True)
result.to_csv("/kaggle/working/data/sentiment_tweet.csv", index=False)

In [ ]:
#importo le librerie utili

import pandas as pd
import os
import json
from google.colab import drive
from tqdm.auto import tqdm   #serve per avere la barra di avanzamento nei loop


#richiamare la cartella condivisa con i csv all'interno

drive.mount ('/content/drive')

Mounted at /content/drive


In [ ]:
#controllo se riesco a leggere i file caricati

df=pd.read_csv('/content/drive/MyDrive/ProjectWorkCondiviso/DB_TWEET_TOTALE_RIPULITO.csv')
df.head()
print(df)

                       created_at        user  \
0       2015-06-27 19:41:36+00:00  2181326676   
1       2015-11-21 13:02:03+00:00  2881595931   
2       2014-04-17 16:38:13+00:00   221896341   
3       2014-04-17 14:30:21+00:00  1839878976   
4       2014-04-17 03:13:30+00:00   300802438   
...                           ...         ...   
285735  2015-12-28 18:30:28+00:00   992182100   
285736  2014-06-11 12:32:39+00:00    68559732   
285737  2015-06-06 12:50:12+00:00   331677065   
285738  2014-02-07 17:39:17+00:00    23059499   
285739  2014-02-07 18:46:04+00:00   992182100   

                                                     text stock_name  \
0       HYG iShares iBoxx High Yield Corporate Bond Fu...          C   
1       $C $ODP $BRCD:\n\nStocks With The Highest Dail...          C   
2                          P$C was the greatest rap group          C   
3       $C Barclays and Citigroup Funded Worst of U.S....          C   
4       RT @Fball_Is_MyDrug: “@OPM1995: im on this 

<ipython-input-11-f2185ed29b86>:3: DtypeWarning: Columns (1,2,3,4,5,6) have mixed types. Specify dtype option on import or set low_memory=False.
  df=pd.read_csv('/content/drive/MyDrive/ProjectWorkCondiviso/DB_TWEET_TOTALE_RIPULITO.csv')


In [ ]:
# importing packages
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt


# plot a Stacked Bar Chart using matplotlib
df.plot(
  x = 'Name',
  kind = 'barh',
  stacked = True,
  title = 'Percentage Stacked Bar Graph',
  mark_right = True)

df_total = df["Studied"] + df["Slept"] + df["Other"]
df_rel = df[df.columns[1:]].div(df_total, 0)*100

for n in df_rel:
    for i, (cs, ab, pc) in enumerate(zip(df.iloc[:, 1:].cumsum(1)[n],
                                         df[n], df_rel[n])):
        plt.text(cs - ab / 2, i, str(np.round(pc, 1)) + '%',
                 va = 'center', ha = 'center')